# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

B. C. Estrada  ->  B. C. Estrada  |  ['B. C. Estrada']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


N. Storm  ->  N. Storm  |  ['N. Storm']
J. Li  ->  J. Li  |  ['J. Li']
S. Vaughan  ->  S. Vaughan  |  ['S. Vaughan']
T. Henning  ->  T. Henning  |  ['T. Henning']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Liu  ->  J. Liu  |  ['J. Liu']
M. Sharma  ->  Y. M. Sharma  |  ['M. Sharma']
Arxiv has 62 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/9 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2501.05521


extracting tarball to tmp_2501.05521...

 done.


Found 185 bibliographic references in tmp_2501.05521/main.bbl.
Issues with the citations
syntax error in line 130: '=' expected
Retrieving document from  https://arxiv.org/e-print/2501.05725


...

/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem{Shu1987}F.~H. {Shu}, F.~C. {Adams}, and S.~{Lizano}, Annual Review of Astronomy and Astrophysics   25, 23 (1987).
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem{1993ApJ...407..806C}A.~{Chokshi}, A.~G.~G.~M. {Tielens}, and D.~{Hollenbach}, \apj   407, 806 (1993).
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem

extracting tarball to tmp_2501.05944... done.
Retrieving document from  https://arxiv.org/e-print/2501.06056


extracting tarball to tmp_2501.06056...

 done.
Retrieving document from  https://arxiv.org/e-print/2501.06075
extracting tarball to tmp_2501.06075... done.


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2501.06075/13hr_UVM2.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'source_table_5lines.tex' from 'tmp_2501.06075/source_table_5lines.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.05521-b31b1b.svg)](https://arxiv.org/abs/2501.05521) | **The $\texttt{MSG}$ model for cloudy substellar atmospheres: A grid of self-consistent substellar atmosphere models with microphysical cloud formation**  |
|| <mark>B. C. Estrada</mark>, et al. |
|*Appeared on*| *2025-01-13*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**|            State-of-the-art JWST observations are unveiling unprecedented views into the atmospheres of substellar objects in the infrared, further highlighting the importance of clouds. Current forward models struggle to fit the silicate clouds absorption feature at ~$10\,\mu$m observed in substellar atmospheres. In the MSG model, we aim to couple the MARCS 1D radiative-convective equilibrium atmosphere model with the 1D kinetic, stationary, non-equilibrium, cloud formation model DRIFT, to create a new grid of self-consistent cloudy substellar atmosphere models with microphysical cloud formation. We aim to test if this new grid is able to reproduce the silicate cloud absorption feature at ~$10\,\mu$m. We model substellar atmospheres with effective temperatures in the range 1200-2500 K and with $\log(g)=4.0$. We compute atmospheric structures that self-consistently account for condensate cloud opacities based on microphysical properties. We present an algorithm based on control theory to help converge such self-consistent models. Synthetic atmosphere spectra are computed for each model to explore the observable impact of the cloud microphysics. We additionally explore the impact of choosing different nucleation species (TiO$_2$ or SiO) and the effect of less efficient atmospheric mixing on these spectra. The new MSG cloudy grid using TiO$_2$ nucleation shows spectra which are redder in the near-infrared compared to the currently known population of substellar atmospheres. We find the models with SiO nucleation, and models with reduced mixing efficiency are less red in the near-infrared. The grid is unable to reproduce the silicate features similar to those found in recent JWST observations and Spitzer archival data. We thoroughly discuss further work that may better approximate the impact of convection in cloud-forming regions and steps that may help resolve the silicate cloud feature.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.05807-b31b1b.svg)](https://arxiv.org/abs/2501.05807) | **The elderly among the oldest: new evidence for extremely metal-poor RR Lyrae stars**  |
|| V. D'Orazi, et al. -- incl., <mark>N. Storm</mark> |
|*Appeared on*| *2025-01-13*|
|*Comments*| *Accepted for publication in Astronomy & Astrophysics; 13 pages. Appendices D-E available on Zenodo*|
|**Abstract**|            We performed a detailed spectroscopic analysis of three extremely metal-poor RR Lyrae stars, exploring uncharted territories at these low metallicities for this class of stars. Using high-resolution spectra acquired with HARPS-N at TNG, UVES at VLT, and PEPSI at LBT, and employing Non-Local Thermodynamic Equilibrium (NLTE) spectral synthesis calculations, we provide abundance measurements for Fe, Al, Mg, Ca, Ti, Mn, and Sr. Our findings indicate that the stars have metallicities of [Fe/H] = -3.40 \pm 0.05, -3.28 \pm 0.02, and -2.77 \pm 0.05 for HD 331986, DO Hya, and BPS CS 30317-056, respectively. Additionally, we derived their kinematic and dynamical properties to gain insights into their origins. Interestingly, the kinematics of one star (HD 331986) is consistent with the Galactic disc, while the others exhibit Galactic halo kinematics, albeit with distinct chemical signatures. We compared the [Al/Fe] and [Mg/Mn] ratios of the current targets with recent literature estimates to determine whether these stars were either accreted or formed in situ, finding that the adopted chemical diagnostics are ineffective at low metallicities ([Fe/H] $\lesssim -$1.5). Finally, the established horizontal branch evolutionary models, indicating that these stars arrive at hotter temperatures on the Zero-Age Horizontal Branch (ZAHB) and then transition into RR Lyrae stars as they evolve, fully support the existence of such low-metallicity RR Lyrae stars. As a consequence, we can anticipate detecting more of them when larger samples of spectra become available from upcoming extensive observational campaigns.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.05913-b31b1b.svg)](https://arxiv.org/abs/2501.05913) | **Dust processing in the terrestrial planet-forming region of the PDS 70 disk**  |
|| Y. Liu, et al. -- incl., <mark>T. Henning</mark>, <mark>G. Perotti</mark> |
|*Appeared on*| *2025-01-13*|
|*Comments*| *12 pages, 5 figures, accepted for publication in SCIENCE CHINA Physics, Mechanics & Astronomy*|
|**Abstract**|            Dust grains in protoplanetary disks are the building blocks of planets. Investigating the dust composition and size, and their variation over time, is crucial for understanding the planet formation process. The PDS 70 disk is so far the only protoplanetary disk with concrete evidence for the presence of young planets. Mid-infrared spectra were obtained for PDS 70 by the Infrared Spectrograph (IRS) on the Spitzer Space Telescope (SST) and the Mid-Infrared Instrument (MIRI) on the James Webb Space Telescope (JWST) in 2007 and 2022, respectively. In this work, we investigate the dust mineralogy through a detailed decomposition of the observed mid-infrared spectra. The results show that both the dust size and crystallinity increased by a factor of about two during the two epochs of observation, indicating evident dust processing in the terrestrial planet-forming region of the PDS 70 disk. The dust size (~0.8 micron) and crystallinity (~6%) in the PDS 70 disk are similar to those of other disks, which implies that the two nascent planets, PDS 70b and PDS 70c located at radial distances of ~22AU and ~34AU, do not have a significant impact on the dust processing in the inner disk. The flux densities at wavelengths longer than ~16 micron measured by JWST/MIRI are only ~60% of those obtained by Spitzer/IRS. Based on self-consistent radiative transfer modeling, we found that such a strong variability in mid-infrared fluxes can be produced by adjustments to the dust density distribution and structure of the inner disk probably induced by planet-disk interaction.         |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.05875-b31b1b.svg)](https://arxiv.org/abs/2501.05875) | **BASSET: Bandpass-Adaptive Single-pulse SEarch Toolkit -- Optimized Sub-Band Pulse Search Strategies for Faint Narrow-Band FRBs**  |
|| J.-H. Cao, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-01-13*|
|*Comments*| *22 pages, 11 figures, submitted to ApJS*|
|**Abstract**|            The existing single-pulse search algorithms for fast radio bursts (FRBs) do not adequately consider the frequency bandpass pattern of the pulse, rendering them incomplete for the relatively narrow-spectrum detection of pulses. We present a new search algorithm for narrow-band pulses to update the existing standard pipeline, Bandpass-Adaptive Single-pulse SEarch Toolkit (BASSET). The BASSET employs a time-frequency correlation analysis to identify and remove the noise involved by the zero-detection frequency band, thereby enhancing the signal-to-noise ratio (SNR) of the pulses. The BASSET algorithm was implemented on the FAST real dataset of FRB 20190520B, resulting in the discovery of additional 79 pulses through reprocessing. The new detection doubles the number of pulses compared to the previously known 75 pulses, bringing the total number of pulses to 154. In conjunction with the pulse calibration and the Markov Chain Monte Carlo (MCMC) simulated injection experiments, this work updates the quantified parameter space of the detection rate. Moreover, a parallel-accelerated version of the BASSET code was provided and evaluated through simulation. BASSET has the capacity of enhancing the detection sensitivity and the SNR of the narrow-band pulses from the existing pipeline, offering high performance and flexible applicability. BASSET not only enhances the completeness of the low-energy narrow-band pulse detection in a more robust mode, but also has the potential to further elucidate the FRB luminosity function at a wider energy scale.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.05886-b31b1b.svg)](https://arxiv.org/abs/2501.05886) | **Estimating variability power spectra of active galaxies from irregular time series**  |
|| M. Lefkir, et al. -- incl., <mark>S. Vaughan</mark> |
|*Appeared on*| *2025-01-13*|
|*Comments*| *21 pages, 16 figures, submitted to MNRAS, code available here: this https URL*|
|**Abstract**|            A common feature of Active Galactic Nuclei (AGN) is their random variations in brightness across the whole emission spectrum, from radio to $\gamma$-rays. Studying the nature and origin of these fluctuations is critical to characterising the underlying variability process of the accretion flow that powers AGN. Random timing fluctuations are often studied with the power spectrum; this quantifies how the amplitude of variations is distributed over temporal frequencies. Red noise variability -- when the power spectrum increases smoothly towards low frequencies -- is ubiquitous in AGN. The commonly used Fourier analysis methods, have significant challenges when applied to arbitrarily sampled light curves of red noise variability. Several time-domain methods exist to infer the power spectral shape in the case of irregular sampling but they suffer from biases which can be difficult to mitigate, or are computationally expensive. In this paper, we demonstrate a method infer the shape of broad-band power spectra for irregular time series, using a Gaussian process regression method scalable to large datasets. The power spectrum is modelled as a power-law model with one or two bends with flexible slopes. The method is fully Bayesian and we demonstrate its utility using simulated light curves. Finally, Ark 564, a well-known variable Seyfert 1 galaxy, is used as a test case and we find consistent results with the literature using independent X-ray data from XMM-Newton and Swift. We provide publicly available, documented and tested implementations in Python and Julia.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.05944-b31b1b.svg)](https://arxiv.org/abs/2501.05944) | **Modeling the Saddle-like GeV-TeV Spectrum of HESS 1809-193: Gamma-Rays Arising from Reverse-Shocked Pulsar Wind Nebula?**  |
|| J. Sun, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-01-13*|
|*Comments*| *11 pages, 5 figures, to appear in ApJ*|
|**Abstract**|            Evolution of pulsar wind nebulae (PWN) could be expected to leave imprints in gamma-rays. We suggest that intriguing GeV-TeV spectral energy distribution (SED) of HESS J1809-193 and Fermi-LAT source J1810.3-1925e is very likely to be the gamma-ray signature of PWN J1809-193 in light of the scenario that the PWN was struck by the reverse shock of the parent supernova remnant. Based on evolutionary theory of PWNe, we consider that, when the PWN was disrupted during collision by the reverse shock, some very high-energy electrons escaped impulsively. The remaining electrons stayed in the relic PWN, which was displaced from the pulsar. The very high-energy part of the remaining electrons were depleted by the strong magnetic field was enhanced by the reverse shock compression in the reverberation stage, leaving the other part of them generating GeV emission. The particles injected from the pulsar after the disruption enter the relic PWN through the newly formed tunnel called the cocoon. The {\gamma}-ray emission from the escaped electrons can account for the TeV spectrum of component A of HESS J1809-193 or the TeV halo, while the electrons remaining after disruption can account for the GeV spectrum of J1810.3-1925e. Thus, combination of contributions from these two populations of electrons naturally reproduces the saddle-like SED of HESS 1809-193 and J1810.31925e from 5 GeV to 30 TeV, together with the spectral hardening around 100 GeV. We also show that the post-disruption injection of electrons can explain the spectrum of the relatively faint gamma-ray emission of component B of HESS J1809-193.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.06056-b31b1b.svg)](https://arxiv.org/abs/2501.06056) | **Periodic Variability of the Central Stars of Planetary Nebulae Surveyed through the Zwicky Transient Facility**  |
|| P. Chen, X. Fang, X. Chen, <mark>J. Liu</mark> |
|*Appeared on*| *2025-01-13*|
|*Comments*| *27 pages, 16 figures; Accepted for publication in ApJ*|
|**Abstract**|            A consensus has been reached in recent years that binarity plays an important role in the formation and evolution of a significant fraction of planetary nebulae (PNe). Utilizing the archived photometric data from the Zwicky Transient Facility survey, we conducted a comprehensive data mining in search for brightness variations in a large sample of Galactic PNe. This effort leads to identification of 39 PNe, whose central stars exhibit periodic variation in light curves. Among these objects, 20 are known binary central stars of PNe, while the remaining 19 are new discoveries. Additionally, we identified 14 PNe with central stars displaying anomalous variation in light curves, as well as eight variables based on the high-cadence photometric data. Among the new discoveries of periodicity, we found compelling evidence of binary systems at the centres of two archetypal quadrupolar PNe. We also report on very peculiar brightness variation observed in the central core of the compact PN NGC6833. Several PNe in our sample deserve follow-up observations, both high-dispersion spectroscopy and high-precision photometry, to reveal the true nature of their central binarity or even multiplicity.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.06075-b31b1b.svg)](https://arxiv.org/abs/2501.06075) | **The ultraviolet luminosity function of star-forming galaxies between redshifts of 0.4 and 0.6**  |
|| M. J. Page, et al. -- incl., <mark>M. Sharma</mark> |
|*Appeared on*| *2025-01-13*|
|*Comments*| *Published in MNRAS*|
|**Abstract**|            We combine ultraviolet imaging of the 13H survey field, taken with the XMM-Newton Optical Monitor telescope (XMM-OM) and the Neil Gehrels Swift Observatory Ultraviolet and Optical Telescope (UVOT) in the UVM2 band, to measure rest-frame ultraviolet 1500A luminosity functions of star-forming galaxies with redshifts between 0.4 and 0.6. In total the UVM2 imaging covers a sky area of 641 square arcmin, and we detect 273 galaxies in the UVM2 image with 0.4<z<0.6. The luminosity function is fit by a Schechter function with best-fit values for the faint end slope alpha = -1.8 +0.4 -0.3 and characteristic absolute magnitude M* = -19.1 +0.3 -0.4. In common with XMM-OM based studies at higher redshifts, our best-fitting value for M* is fainter than previous measurements. We argue that the purging of active galactic nuclei from the sample, facilitated by the co-spatial X-ray survey carried out with XMM-Newton is important for the determination of M*. At the brightest absolute magnitudes (M1500<-18.5) the average UV colour of our galaxies is consistent with that of minimal-extinction local analogues, but the average UV colour is redder for galaxies at fainter absolute magnitudes, suggesting that higher levels of dust attenuation enter the sample at absolute magnitudes somewhat fainter than M*.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2501.05521/./figures/p_vs_tau_TiO2.png', 'tmp_2501.05521/./figures/cloud_comp_TiO2.png', 'tmp_2501.05521/./figures/elem_TiO2.png', 'tmp_2501.05521/./figures/spectra_g4.00_TiO2_A.png', 'tmp_2501.05521/./figures/spectra_g4.00_TiO2_B.png', 'tmp_2501.05521/./figures/spectra_g4.00_TiO2_D.png', 'tmp_2501.05521/./figures/spectra_g4.00_TiO2_E.png', 'tmp_2501.05521/./figures/pt_profiles_TiO2_vs_SiO.png', 'tmp_2501.05521/./figures/cloud_size_nd_TiO2_vs_SiO.png', 'tmp_2501.05521/./figures/nucleation_rate_TiO2_vs_SiO.png']
copying  tmp_2501.05521/./figures/p_vs_tau_TiO2.png to _build/html/
copying  tmp_2501.05521/./figures/cloud_comp_TiO2.png to _build/html/
copying  tmp_2501.05521/./figures/elem_TiO2.png to _build/html/
copying  tmp_2501.05521/./figures/spectra_g4.00_TiO2_A.png to _build/html/
copying  tmp_2501.05521/./figures/spectra_g4.00_TiO2_B.png to _build/html/
copying  tmp_2501.05521/./figures/spectra_g4.00_TiO2_D.png to _build/html/
copying  tmp_2501.05521/./figures/sp

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\dal}[1]{\textcolor{purple}{#1^{(DAL)}}}$
$\newcommand{\Teff}{T_{\rm{eff}}}$
$\newcommand{\PT}{P_{\mathrm{gas}}-T_{\mathrm{gas}}}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# The \texttt{MSG} model for cloudy substellar atmospheres:: A grid of self-consistent substellar atmosphere models with microphysical cloud formation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2501.05521-b31b1b.svg)](https://arxiv.org/abs/2501.05521)<mark>Appeared on: 2025-01-13</mark> -  _Accepted for publication in A&A_

</div>
<div id="authors">

<mark>B. C. Estrada</mark>, et al.

</div>
<div id="abstract">

**Abstract:** State-of-the-art JWST observations are unveiling unprecedented views into the atmospheres of substellar objects in the infrared, further highlighting the importance of clouds. Current forward models struggle to fit the silicate clouds absorption feature at $\sim10 \mu$ m observed in substellar atmospheres. In the \texttt{MSG} model, we aim to couple the \texttt{MARCS} 1D radiative-convective equilibrium atmosphere model with the 1D kinetic, stationary, non-equilibrium, cloud formation model \texttt{DRIFT} , also known as \texttt{StaticWeather} , to create a new grid of self-consistent cloudy substellar atmosphere models with microphysical cloud formation. We aim to test if this new grid is able to reproduce the silicate cloud absorption feature at $\sim10 \mu$ m. We model substellar atmospheres with effective temperatures in the range $T_{\rm{eff}}$ = 1200-2500 K and with $\log(g)=4.0$ . We compute atmospheric structures that self-consistently account for condensate cloud opacities based on microphysical properties. We present an algorithm based on control theory to help converge such self-consistent models. Synthetic atmosphere spectra are computed for each model to explore the observable impact of the cloud microphysics. We additionally explore the impact of choosing different nucleation species ( $\ce{TiO2}$ or $\ce{SiO}$ ) and the effect of less efficient atmospheric mixing on these spectra. The new \texttt{MSG} cloudy grid using $\ce{TiO2}$ nucleation shows spectra which are redder in the near-infrared compared to the currently known population of substellar atmospheres. We find the models with $\ce{SiO}$ nucleation, and models with reduced mixing efficiency are less red in the near-infrared. We present a new grid of \texttt{MSG} models for cloudy substellar atmospheres that include cloud-radiative feedback from microphysical clouds. The grid is unable to reproduce the silicate features similar to those found in recent JWST observations and _Spitzer_ archival data. We thoroughly discuss further work that may better approximate the impact of convection in cloud-forming regions and steps that may help resolve the silicate cloud feature.

</div>

<div id="div_fig1">

<img src="tmp_2501.05521/./figures/p_vs_tau_TiO2.png" alt="Fig8.1" width="33%"/><img src="tmp_2501.05521/./figures/cloud_comp_TiO2.png" alt="Fig8.2" width="33%"/><img src="tmp_2501.05521/./figures/elem_TiO2.png" alt="Fig8.3" width="33%"/>

**Figure 8. -** **Left:** The optical depth of the cloud (orange), gas (blue), and the total (green) throughout the atmosphere at $\lambda \approx 1.1  \mu$m (solid curves) and $\lambda \approx 10.0  \mu$m (dashed curves), for a \texttt{MSG} model with $TiO_2$ nucleation, at 1500 K and $\log(g)=4.0$. The $\tau=1.0$ level is shown for reference (dot-dash red curve). **Middle and right:** Composition of the cloud particles in units of volume fractions $V_s/V_{\mathrm{tot}}$(middle) and the relative gas-phase element depletions $\epsilon_i/ \epsilon_i^0$(right) for the same model as in the left panel.  (*fig:cloud_comp_TiO2*)

</div>
<div id="div_fig2">

<img src="tmp_2501.05521/./figures/spectra_g4.00_TiO2_A.png" alt="Fig9.1" width="25%"/><img src="tmp_2501.05521/./figures/spectra_g4.00_TiO2_B.png" alt="Fig9.2" width="25%"/><img src="tmp_2501.05521/./figures/spectra_g4.00_TiO2_D.png" alt="Fig9.3" width="25%"/><img src="tmp_2501.05521/./figures/spectra_g4.00_TiO2_E.png" alt="Fig9.4" width="25%"/>

**Figure 9. -** Synthetic spectra of \texttt{MSG} models with $TiO_2$ nucleation at $T_{\rm{eff}}$$=$ 2400 K (blue), 2100 K (orange), 1800 K (green) and 1500 K (pink), with $\log(g)=4.0$, and the respective cloud-free spectra at the same $T_{\rm{eff}}$ and $\log(g)$ in grey. The emergent fluxes are normalised with respect to the cloud-free \texttt{MSG} spectra, i.e. $F_{\mathrm{norm}}(\lambda) = F_{\mathrm{cloudy}}(\lambda) / F^{\mathrm{max}}_{\mathrm{cloud-free}}$, where $F_{\mathrm{norm}}$ is the normalised flux, $F_{\mathrm{cloudy}}$ the flux from the cloudy model, and $F^{\mathrm{max}}_{\mathrm{cloud-free}}$ the maximum flux from the cloud-free model within the full wavelength range considered ($\sim$ 0.4$ \mu$m - 20.0$ \mu$m). An arbitrary offset is added for clarity. In the top-left plots we show the NIR range, in the top-right the Y and J bands, in the bottom-left the MIR and the bottom-right the thermal infrared. Important absorbers are respectively labelled. (*fig:spectra_TiO2*)

</div>
<div id="div_fig3">

<img src="tmp_2501.05521/./figures/pt_profiles_TiO2_vs_SiO.png" alt="Fig11.1" width="33%"/><img src="tmp_2501.05521/./figures/cloud_size_nd_TiO2_vs_SiO.png" alt="Fig11.2" width="33%"/><img src="tmp_2501.05521/./figures/nucleation_rate_TiO2_vs_SiO.png" alt="Fig11.3" width="33%"/>

**Figure 11. -** **Left**: Pressure-temperature profiles for \texttt{MSG} models at $T_{\rm{eff}}=$ 1500 K and $\log(g)=4.0$ with \ce{TiO2} nucleation (blue dashed curve) and \ce{SiO} nucleation (orange solid curve). Convective zones are plotted with thicker lines while radiative zones are plotted with thinner lines. **Middle:** The average cloud particle size (solid curves) and the cloud particle number density (dashed curves) for the models shown on the left. **Right:** The nucleation rates for \texttt{MSG} models at $T_{\rm{eff}}$$=$ 1500 K and $\log(g)=4.0$ with \ce{TiO2} nucleation (blue curve) and \ce{SiO} nucleation (orange curve). (*fig:pt_SiO*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2501.05521"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\teff}{T_{\text{eff}}}$</div>



<div id="title">

# The elderly among the oldest: new evidence for extremely metal-poor RR Lyrae stars$\thanks{Based on observations acquired at the Telescopio Nazionale Galileo under program A43DDT3, on a DDT program with PEPSI at LBT (2021-2022, PI Crestani) and on VLT ESO programs 69.C-0423(A) and 165.N-0276(A).}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2501.05807-b31b1b.svg)](https://arxiv.org/abs/2501.05807)<mark>Appeared on: 2025-01-13</mark> -  _Accepted for publication in Astronomy & Astrophysics; 13 pages. Appendices D-E available on Zenodo_

</div>
<div id="authors">

V. D'Orazi, et al. -- incl., <mark>N. Storm</mark>

</div>
<div id="abstract">

**Abstract:** We performed a detailed spectroscopic analysis of three extremely metal-poor RR Lyrae stars, exploring uncharted territories at these low metallicities for this class of stars. Using high-resolution spectra acquired with HARPS-N at TNG, UVES at VLT and PEPSI at LBT, and employing non-LTE spectral synthesis calculations, we provide abundance measurements for Fe, Al, Mg, Ca, Ti, Mn, and Sr.  Our findings indicate that the stars have metallicities of [ Fe/H ] = $-3.40 \pm 0.05$ , $-3.28 \pm 0.02$ , and $-2.77 \pm 0.05$ for HD 331986, DO Hya, and BPS CS 30317-056, respectively. Additionally, we derived their kinematic and dynamical properties to gain insights into their origins. Interestingly, the kinematics of one star (HD 331986) is consistent with the Galactic disc, while the others exhibit Galactic halo kinematics, albeit with distinct chemical signatures. We compared the [ Al/Fe ] and [ Mg/Mn ] ratios of the current targets with recent literature estimates to determine whether these stars were either accreted or formed $in situ$ , finding that the adopted chemical diagnostics are ineffective at low metallicities  ( [ Fe/H ] $\lesssim -$ 1.5).  Finally, the established horizontal branch evolutionary models, indicating that these stars are born hotter at the Zero-Age Horizontal Branch (ZAHB) and then transition into RR Lyrae stars as they evolve, fully support the existence of such low-metallicity RR Lyrae stars. As a consequence, we can anticipate detecting more of them when larger samples of spectra become available from upcoming extensive observational campaigns.

</div>

<div id="div_fig1">

<img src="tmp_2501.05807/./synthesys.jpeg" alt="Fig4" width="100%"/>

**Figure 4. -** Comparison of the observed (grey) and synthetic (green) spectra. For HD 331986 (upper panel), we present the upper limit synthesis, while for DO Hya (middle panel) and BPS CS 30317-056 (lower panel), we display the best-fit results along with a variation of $\pm 0.2 \mathrm{dex}$(shaded areas). (*fig:synth*)

</div>
<div id="div_fig2">

<img src="tmp_2501.05807/./CMD_VBI.png" alt="Fig6.1" width="50%"/><img src="tmp_2501.05807/./CMD_KVK.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** Left -- Optical $B$-$I$, $V$ CMD showing the location
    of very metal-poor RRLs. The solid lines display predicted Zero Age Horizontal
    Branches at different chemical compositions (see labelled values), while the
    dashed lines the central helium exhaustion for the same chemical compositions
     ([Pietrinferni, Hidalgo and Cassisi 2021]()) .
    Squares and triangles mark different stellar masses along the ZAHB and
    end-of-helium. The almost vertical lines display the blue (hot) and the red (cool)
    edge of the predicted RRL instability strip for Z=1$\times$10$^{-5}$ specifically
    computed for this investigation (see text for more details).
    Right -- Same as the left, but for the optical-NIR ($V$-$K$, $K$) CMD. (*fig:opt_nir_cmd*)

</div>
<div id="div_fig3">

<img src="tmp_2501.05807/./MgMn_AlFe_GALAHDR4_two_panels.jpeg" alt="Fig5" width="100%"/>

**Figure 5. -** [Mg/Mn] ratios as a function of [Al/Fe] for our three RRLs (filled markers) along with literature estimates, including GALAH DR4 \citep[][density plots in this figure]{buder2024} and $Loki$(empty circles) by [Sestito, Fernandez-Alvar and Brooks (2024)](). (*fig:MgMn_AlFe*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2501.05807"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\aap}{A\&A, }$
$\newcommand{\apj}{ApJ, }$
$\newcommand{\aj}{AJ, }$
$\newcommand{\apjl}{ApJ, }$
$\newcommand{\apjs}{ApJS, }$
$\newcommand{\mnras}{MNRAS, }$
$\newcommand{\icarus}{Icarus, }$
$\newcommand{\araa}{ARA\&A, }$
$\newcommand{\nat}{Nature, }$
$\newcommand{\nar}{New Astronomy Reviews, }$
$\newcommand{\pasj}{PASJ, }$
$\newcommand{\pasp}{PASP, }$
$\newcommand{\thesection}{Appendix}$
$\newcommand\cpartlineleft{#1}$
$\newcommand{\@}{cpartlineleft#1,#2}$</div>



<div id="title">

# Dust processing in the terrestrial planet-forming region of the PDS 70 disk

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2501.05913-b31b1b.svg)](https://arxiv.org/abs/2501.05913)<mark>Appeared on: 2025-01-13</mark> -  _12 pages, 5 figures, accepted for publication in SCIENCE CHINA Physics, Mechanics & Astronomy_

</div>
<div id="authors">

Y. Liu{yliu@swjtu.edu.cn}, et al. -- incl., <mark>G. Perotti</mark>

</div>
<div id="abstract">

**Abstract:** Dust grains in protoplanetary disks are the building blocks of planets. Investigating the dust composition and size, and their variation over time, is crucial for understanding the planet formation process. The PDS 70 disk is so far the only protoplanetary disk with concrete evidence for the presence of young planets. Mid-infrared spectra were obtained for PDS 70 by the Infrared Spectrograph (IRS) on the Spitzer Space Telescope (SST) and the Mid-Infrared Instrument (MIRI) on the James Webb Space Telescope (JWST) in 2007 and 2022, respectively. In this work, we investigate the dust mineralogy through a detailed decomposition of the observed mid-infrared spectra. The results show that both the dust size and crystallinity increased by a factor of about two during the two epochs of observation, indicating evident dust processing in the terrestrial planet-forming region of the PDS 70 disk. The dust size ( ${\sim} 0.8 \mu{\rm m}$ ) and crystallinity ( ${\sim} 6\%$ ) in the PDS 70 disk are similar to those of other disks, which implies that the two nascent planets, PDS 70b and PDS 70c located at radial distances of ${\sim} 22 {\rm AU}$ and ${\sim} 34 {\rm AU}$ , do not have a significant impact on the dust processing in the inner disk. The flux densities at $\lambda{\gtrsim} 16 \mu{\rm m}$ measured by JWST/MIRI are only 60 \% of those obtained by Spitzer/IRS. Based on self-consistent radiative transfer modeling, we found that such a strong variability in mid-infrared fluxes can be produced by adjustments to the dust density distribution and structure of the inner disk probably induced by planet-disk interaction.

</div>

<div id="div_fig1">

<img src="tmp_2501.05913/./figure2.png" alt="Fig3" width="100%"/>

**Figure 3. -** Comparison between model spectra (black line) and observed spectra (red line) of PDS 70. The upper panel shows the IRS spectrum, whereas the MIRI spectrum is shown in the bottom panel. For the model spectra, we also mark the contribution by the underlying continuum and dust emission from the disk surface layer, see Eq. \ref{eq:newFnu}. The vertical bars on the IRS and MIRI spectra refer to the observational uncertainties. Note that the uncertainties of the IRS spectrum are barely seen. The vertical bar shown in the upper left corner of each panel indicates the finally-converged uncertainty obtained from the iterative fitting process, see Sect. \ref{sec:fitapp}. (*fig:fluxCom*)

</div>
<div id="div_fig2">

<img src="tmp_2501.05913/./figure4.png" alt="Fig5" width="100%"/>

**Figure 5. -** Radiative transfer models of PDS 70. _ Left panel: _ dust surface densities of model A (grey line) and model B (black line). Model parameters are given in Table \ref{tab:paras}. Note that the surface densities of the outer disk for both models are identical. _ Middle panel: _ comparison of infrared spectrum between model and observation. The IRS data is indicated with the red line, whereas the blue line represents the MIRI spectrum. _ Right panel: _ comparison between the model SED and photometry from  optical to millimeter wavelengths. Data points are collected from Gregorio-Hetem et al. Gregorio-Hetem1992, Hashimoto et al. Hashimoto2012, Benisty et al. Benisty2021, and Facchini et al. Facchini2021. (*fig:surdens*)

</div>
<div id="div_fig3">

<img src="tmp_2501.05913/./figure1.png" alt="Fig2" width="100%"/>

**Figure 2. -** Mass absorption coefficients of amorphous silicates of olivine type (left panel) and pyroxene type (middle panel), and crystalline forsterite (right panel). The solid lines show the results for the grain size being $0.1 \mu{\rm m}$. The dashed lines indicate the results for $2 \mu{\rm m}$-sized dust grains. The dotted lines display the results for $5 \mu{\rm m}$-sized dust grains. The results are calculated using the theory of distribution of hollow spheres Min2005. (*fig:kappa*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2501.05913"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

305  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
